In [ ]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

from aind_vr_foraging_analysis.utils.parsing import data_access
import aind_vr_foraging_analysis.data_io as data_io
import os
import json

# Plotting libraries
import pandas as pd
import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)


foraging_figures = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents'

# **One session exploration**

In [ ]:
date_string = "2024-4-1"
mouse_list = ['754570','754579','754567','754580','754559','754560','754577',
              '754566','754571','754572','754573','754574','754575', 
              '754582','745302','745305','745301',
              
              "715866", "713578", "707349", "716455", 
              "716458","715865","715869","713545","715867",
              "715870","694569", 
              
              '754574', '789914', '789915', '789923', '789917', 
               '789913', '789909', '789910', '789911', '789921', 
               '789918', '789919', '789907', '789903', '789925', 
               '789924', '789926', '789908']

experiment_list = { 1: 'pilot',
                    2 : 'volume_manipulation',
                   3: 'global_reward_rate_patches', 
                   4: 'global_reward_rate_distance_friction',
                   5 : 'learning_reversals'}

In [ ]:
def parse_summary_session(session_path):
    df = {}
    data = {}
    # Work around the change in the folder structure
    if "behavior" in os.listdir(session_path):
        session_path_behavior = session_path / "behavior"
    else:
        session_path_behavior = session_path
        
    if "other" in os.listdir(session_path):
        session_path_config = session_path / "other"
    else:
        session_path_config = session_path
        
        # Load config old version
    if "config.json" in os.listdir(session_path_config):
        with open(str(session_path_config) + r"\config.json", "r") as json_file:
            data["config"] = json.load(json_file)
    elif "Logs" in os.listdir(session_path_behavior):
        data["config"] = data_io.ConfigSource(path=session_path_behavior / "Logs", name="config", autoload=True)
        
    rig = data['config'].streams.rig_input.data
    task_logic = data['config'].streams.tasklogic_input.data
    
    df['rig'] = rig['rig_name']
    try:
        df['stage'] = task_logic['stage_name']
    except:
        df['stage'] = 'no_stage'
    df['VAST'] = str(session_path)[10:]
    df['s3'] = str(session_path)[27:]
    return df

In [ ]:
df = pd.DataFrame(columns=['mouse', 'rig', 'stage', 'VAST', 's3'])
for mouse in mouse_list:
    print(f"Loading {mouse}...")
    session_paths = data_access.find_sessions_relative_to_date(
        mouse=mouse,
        date_string=date_string,
        when='on_or_after',
    )

    for session_path in session_paths:
        try:
            new_row = parse_summary_session(session_path)
            if int(mouse) > 760000:
                batch = 5
            elif  int(mouse) < 760000 and int(mouse) > 740000 :
                batch = 4
            elif int(mouse) < 740000 and int(mouse) > 700000:
                batch = 3
            elif int(mouse) < 700000 and int(mouse) > 690000:
                batch = 2
            else:
                batch = 1
                
            new_row['experiment'] = experiment_list[batch]
            new_row['batch'] = batch
            new_row['mouse'] = mouse
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        except Exception as e:
            print(f"Error loading {session_path.name}: {e}")

In [ ]:
data_path = r'../../../data/'
df.to_csv(os.path.join(data_path, "summary_sessions.csv"), index=False)